Este es un cuaderno simple para hacer ACP en espectros SDSS e imágenes de galaxias.

Acompaña al Capítulo 7 del libro (2 de 4).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman.

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt

from sklearn import preprocessing, decomposition
%matplotlib inline

import skimage
from skimage.transform import resize, rescale
from skimage import io

### Reducción de dimensionalidad

El análisis de componentes principales (ACP), o PCA por sus siglas en inglés, y algoritmos similares se utilizan para la reducción de la dimensionalidad en las ciencias con uso intensivo de datos.

El objetivo principal del ACP lineal es encontrar las combinaciones lineales de características más representativas, de modo que cada elemento de un conjunto de datos pueda expresarse como la superposición (suma) de algunos vectores sobresalientes en el espacio de características (no es necesario que sean elementos de un conjunto de datos). En el ACP lineal más simple, los componentes principales son los vectores propios de la matriz de covarianza del conjunto de datos.

Si el número de componentes es muy bajo (por ejemplo, 2 o 3, ACP u otros métodos de reducción de dimensionalidad permiten visualizar un conjunto de datos de alta dimensión como un gráfico 2D o 3D. Scikit-learn tiene métodos para calcular PCA y varias variantes. ACP Clásico tiene una complejidad difícil: $\mathcal{O}[N^3].$

### Veamos un ejemplo con espectros de galaxias de
https://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/tutorial/astronomy/dimensionality_reduction.html#sdss-spectral-data

In [ ]:
data = np.load('../data/spec4000_corrected.npz')

In [ ]:
wavelengths = data['wavelengths'] #wavelengts en inglés Longitud de onda
X = data['X']
y = data['y']
labels = data['labels'].astype('str')

In [ ]:
X.shape # Forma de la matriz

In [ ]:
y

In [ ]:
labels #No nos importan.

### Podemos graficar algunos ejemplos representativos de cada clase, solo para tener una idea de qué tipo de espectros hay en el conjunto de datos.


In [ ]:
plt.figure(figsize=(10,5))

for i_class in (2, 3, 4, 5, 6):
    i = np.where(y == i_class)[0][0]
    l = plt.plot(wavelengths, X[i] + 20 * i_class)
    c = l[0].get_color()
    plt.text(6800, 2 + 20 * i_class, labels[i_class], color=c)

plt.subplots_adjust(hspace=0)
plt.xlabel('Longitud de onda (Angstroms)') 
plt.ylabel('flujo + constante')# Flujo
plt.title('Muestra de Espectros'); # QS

### Nuestro conjunto de datos original tiene 4000 objetos y 1000 características.

Intentaremos representarlo con una cantidad variable de componentes.



In [ ]:
#  Realizar ACP

scaler = preprocessing.StandardScaler() #¡Es importante que los datos estén centrados!

Xn = scaler.fit_transform(X) #Este es un procedimiento de estandarización.

pca_50 = decomposition.PCA(n_components=50, random_state=0)# PCA por sus siglas en inglés

pca_100 = decomposition.PCA(n_components=100, random_state=0)

pca_1000 = decomposition.PCA(n_components=1000, random_state=0)

X_proj_50 = pca_50.fit_transform(Xn) #el conjunto de datos proyectados: vive en un nuevo espacio de funciones con 4000 objetos y 50 características

X_proj_100 = pca_100.fit_transform(Xn) #el conjunto de datos proyectados: vive en un nuevo espacio de funciones con 4000 objetos y 100 características

X_proj_1000 = pca_1000.fit_transform(Xn) #el conjunto de datos proyectados: vive en un nuevo espacio de funciones con 4000 objetos y 1000 características


### Registro de aprendizaje
    
¿Fue redundante el proceso anterior?
<br>

<details><summary><b>Haga clic aquí para la respuesta</b></summary>
<p>
    
```
¡Sí! Los ACP se calculan de forma iterativa, siguiendo el mismo procedimiento, por lo que los primeros 50 componentes serán siempre los mismos, por muchos que generemos. Podríamos haber generado 1000 y luego mirar los primeros 50, 100 o 1000.
```
    
</p>
</details>


In [ ]:
#----------------------------------------------------------------------
#
#  graficar espectros propios de PCA
#

plt.figure()

l = plt.plot(wavelengths, pca_50.mean_ - 0.15)
c = l[0].get_color()
plt.text(7000, -0.16, "media", color=c) 

# En ACP lineal, el primer vector propio es siempre la media, 
# y los primeros n componentes son siempre los mismos

for i in range(4):
    
    l = plt.plot(wavelengths, pca_50.components_[i] + 0.15 * i)
    
    l = plt.plot(wavelengths, pca_100.components_[i] + 0.15 * i, linestyle = '-.')
    
    c = l[0].get_color()
    
    plt.text(7000, -0.01 + 0.15 * i, "componente %i" % (i + 1), color=c)

    plt.ylim(-0.2, 0.6)
    
plt.xlabel('longitud de onda (Angstroms)')
plt.ylabel('flujo escalado + constante')
plt.title('Espectro medio y espectros propios')

plt.show()


Podemos estimar la contribución de cada componente usando la propiedad "razón de varianza explicada".

Estos son simplemente los valores propios de la matriz de covarianza. Su suma acumulada da la razón de varianza explicada (creciente progresivamente).


In [ ]:
pca_50.explained_variance_ratio_ 

In [ ]:
pca_1000.explained_variance_ratio_[-10:]

Podemos interpretar los vectores propios como la "base" que explica la mayor parte de la variabilidad de los datos.

¿Cómo podemos saber si esto funciona? Hagamos ingeniería inversa del proceso:

In [ ]:
Xrec_50 = pca_50.inverse_transform(X_proj_50) 

Xrec_100 = pca_100.inverse_transform(X_proj_100)

Xrec_1000 = pca_1000.inverse_transform(X_proj_1000)

In [ ]:
plt.figure(figsize=(10,5))

for i in range(4,8):
    plt.subplot(2,2,i-3)
    #plt.plot(wavelengths, Xn[i], label = 'orig', c = 'k')
    #plt.plot(wavelengths, Xrec_50[i], '--', label = 'new, 50 PCs', c = 'g')
    #plt.plot(wavelengths, Xrec_100[i], '--', label = 'new, 100 PCs', c = 'b')
    #plt.plot(wavelengths, Xrec_1000[i], '--', label = 'new, 1000 PCs', c = 'r')
    #plt.plot(wavelengths, (Xrec_50[i]-Xn[i])/Xn[i], '--', label = '% diff 50', c = 'g')
    plt.plot(wavelengths, (Xrec_100[i]-Xn[i])/Xn[i], '--', label = 'diff 100', c = 'b')
    #plt.plot(wavelengths, (Xrec_1000[i]-Xn[i])/Xn[i], '-.', label = 'diff 1000', c = 'k')
    plt.ylim(-0.5,0.5)
    plt.legend();
plt.xlabel('longitud de onda (Angstroms)');


### Registro de aprendizaje
    
¿Cómo espera que cambien los gráficos anteriores, si graficamos 1000 componentes en lugar de 100?

<br>

<details><summary><b>Haga clic aquí para la respuesta</b></summary>
<p>
    
```
 La diferencia entre los espectros originales y el PCA con la misma cantidad de componentes que las entradas debe ser cero (o insignificante); en realidad, esta es una buena prueba de verificación.
```
    

### Pregunta: ¿cómo podemos saber cuál es un buen número de componentes?

Para tener una idea, podemos graficar la propiedad "razón de varianza explicada" ( en inglés "explained_variance_ratio") de la descomposición ACP. Se parece mucho al método del codo, pero al revés; en particular, la varianza explicada por N componentes siempre aumenta con N, pero suele haber un punto después del cual los rendimientos tienden a disminuir.

In [ ]:
plt.plot(np.cumsum(pca_1000.explained_variance_ratio_))
plt.xlabel('número de componentes')
plt.ylabel('varianza explicada acumulada');
plt.xlim(0,20)

### Registro de aprendizaje
    
¿Qué número de componentes recomendaría para el caso anterior?
<br>

<details><summary><b>Haga clic aquí para la respuesta</b></summary>
<p>
   
    
```
Si solo nos basamos en la varianza explicada, parece que 5 o 10 componentes son suficientes, ¡pero esto puede NO ser lo suficientemente bueno para la ciencia que tenemos que hacer! La diferencia porcentual con los originales muestra que incluso para 50 o 100 componentes, todavía hay áreas (líneas de emisión/absorción, en particular) donde las diferencias son notables. Si son o no importantes realmente depende del caso de uso. En resumen: ¡No confíe únicamente en la varianza explicada!
```
    
</p>
</details>



See also:
    
https://arxiv.org/abs/2012.00066

### Ejemplo de ACP de Kernel

In [ ]:
pca_50 = decomposition.PCA(n_components=50, random_state=0)

In [ ]:
kpca_50 = decomposition.KernelPCA(n_components=50, \
                kernel = 'rbf', gamma = 0.2, eigen_solver = 'dense', 
                                  fit_inverse_transform = True, random_state=0)

In [ ]:
X_kproj_50 = kpca_50.fit_transform(Xn);

In [ ]:
X_proj_50.shape

In [ ]:
X_krec_50 = kpca_50.inverse_transform(X_kproj_50) #transformación más compleja

In [ ]:
plt.figure(figsize=(10,5))

for i in range(4,8):
    plt.subplot(2,2,i-3)
    #plt.plot(wavelengths, Xn[i], label = 'orig', c = 'k')
    #plt.plot(wavelengths, X_krec_50[i], '--', label = 'new, 50 kPCs', c = 'g')
    plt.plot(wavelengths, (X_krec_50[i]-Xn[i])/Xn[i], '--', label = '% diff k50', c = 'k')
    plt.ylim(-0.1,0.1)
    plt.legend()
plt.xlabel('longitud de onda(Angstroms)');


In [ ]:
X_kproj_50.shape

In [ ]:
kpca_50.lambdas_ #valores propios - Notese el cambio

In [ ]:
alphas = kpca_50.alphas_ #vectores propios

In [ ]:
#Comparar con esta implementación ( en inglés)
#From https://sebastianraschka.com/Articles/2014_kernel_pca.html

from scipy.spatial.distance import pdist, squareform
from scipy.linalg import eigh

def stepwise_kpca(X, gamma, n_components):
    """
    Implementación de un ACP de kernel PCA con una función de base radial (RBF por sus siglas en inglés).

    Argumentos:
        X: Un conjunto de datos MxN como matriz NumPy donde las muestras se almacenan como filas (M),
           y los atributos definidos como columnas (N).
        gamma: Un parámetro libre (coeficiente) para el kernel RBF.
        n_components: El número de componentes a devolver.

    Devuelve los k autovectores (alfas) que corresponden a los k más grandes valores propios (lambdas).      
    """
    # Cálculo de las distancias euclidianas al cuadrado para cada par de puntos
    # en el conjunto de datos dimensionales MxN.
    sq_dists = pdist(X, 'sqeuclidean')

    # Convertir las distancias por pares en una matriz MxM simétrica.
    mat_sq_dists = squareform(sq_dists)

    # Cálculo de la matriz kernel MxM.
    K = np.exp(-gamma * mat_sq_dists)

    # Centrado de la matriz kernel NxN simétrica.
    N = K.shape[0]
    one_n = np.ones((N,N)) / N
    K_norm = K - one_n.dot(K) - K.dot(one_n) + one_n.dot(K).dot(one_n)

    # Obtención de valores propios en orden descendente con su correspondiente
    # vectores propios de la matriz simétrica.
    eigvals, eigvecs = eigh(K_norm)

    # Obtención de los i vectores propios (alfas) que corresponden a los i valores propios más altos (lambdas).
    
    alphas = np.column_stack((eigvecs[:,-i] for i in range(1,n_components+1)))
    
    lambdas = [eigvals[-i] for i in range(1,n_components+1)]

    return alphas, lambdas

### Ahora echemos un vistazo a las imágenes.

Este conjunto de datos está compuesto por 200 imágenes seleccionadas aleatoriamente del desafío Kaggle Galaxy Zoo:

https://www.kaggle.com/c/galaxy-zoo-the-galaxy-challenge

El siguiente código visualiza los primeros 25 objetos en su conjunto de datos. Puede ejecutarlo para obtener una vista de las primeras 25 galaxias. Nota: es posible que reciba un mensaje de error, en este caso, consulte aquí (en inglés):

https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image

In [ ]:
#Takes <1 minuto

images = []
for i in range(200):
    img =skimage.io.imread('../data/galaxy_images/Image_'+str(i)+'.png')
    img_resized = resize(img,(100,100))
    length = np.prod(img_resized.shape)
    img_resized = np.reshape(img_resized,length)
    images.append(img_resized)
    
images = np.vstack(images)

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(ax.shape[0]):

    img = skimage.io.imread('../data/galaxy_images/Image_'+str(i)+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])


### Aquí, hacemos la descomposición ACP en cada uno de los canales RGB (rojo, verde y azul) por separado. No estamos seguro de si es óptimo.

In [ ]:
r_images = images.reshape(200, -1,  3)[:,:,0]

In [ ]:
r_images.shape

In [ ]:
#Ejecutar ACP en las imágenes

estimator = decomposition.PCA(n_components=100)

r_images_PCA = estimator.fit_transform(r_images)

In [ ]:
#Esto nos indica de la reducción de dimensionalidad que hemos hecho
r_images_PCA.shape

In [ ]:
components = estimator.components_

### Podemos trazar los primeros 50 componentes.

In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(12, 6),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow((estimator.components_[i].reshape(100, 100)), cmap='bone')

### Registro de aprendizaje
    
A partir de esta gráfica, ¿cuál creerías que es un número óptimo de componentes?

<br>

<details><summary><b>Haga clic aquí para la respuesta</b></summary>
<p>
    
```
Es difícil saberlo, pero quizás después de ~30 haya muy poca estructura en las imágenes propias.```
    
</p>
</details>




Podemos usar la relación de varianza explicada para ver si hay un número óptimo obvio de componentes.

In [ ]:
plt.plot(np.cumsum(estimator.explained_variance_ratio_))
plt.xlabel('número de componentes')
plt.ylabel('varianza explicada acumulada');

### Conclusión: de nuevo, no es obvio, pero tal vez 25-30.

### Reconstruyamos ahora las imágenes originales.

In [ ]:
r_projected = estimator.inverse_transform(r_images_PCA)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(15, 5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(r_images[i].reshape(100, 100), cmap='gray')
    ax[1, i].imshow(r_projected[i].reshape(100, 100), cmap='gray')
#diff    ax[1, i].imshow((r_projected[i] - r_images[i]).reshape(100, 100), cmap='gray')

Podemos hacerlo para los tres canales a la vez y luego unirlos:

In [ ]:
estimator = decomposition.PCA(n_components=100) 

r_images = images.reshape(200, -1,  3)[:,:,1]                     
estimator.fit(r_images)
r_images_PCA = estimator.fit_transform(r_images)
r_projected = estimator.inverse_transform(r_images_PCA)

g_images = images.reshape(200, -1,  3)[:,:,1]                     
estimator.fit(g_images)
g_images_PCA = estimator.fit_transform(g_images)
g_projected = estimator.inverse_transform(g_images_PCA)

b_images = images.reshape(200, -1,  3)[:,:,2]                     
estimator.fit(b_images)
b_images_PCA = estimator.fit_transform(b_images)
b_projected = estimator.inverse_transform(b_images_PCA)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 5, figsize=(50, 20),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(5):
    ax[0, i].imshow((np.dstack([r_images[i].reshape(100, 100)*255, g_images[i].reshape(100, 100)*255, 
        b_images[i].reshape(100,100)*255]).astype(np.uint8)))
    ax[1, i].imshow((np.dstack([r_projected[i].reshape(100, 100)*255, g_projected[i].reshape(100, 100)*255, 
        b_projected[i].reshape(100,100)*255]).astype(np.uint8)))

## Conclusiones

Las técnicas de reducción de la dimensionalidad son útiles tanto para desarrollar la comprensión de lo que hay en los datos como para hacer que los tamaños sean más manejables.

¡El agrupamiento y la reducción de dimensionalidad se superponen mucho! Por ejemplo en el Ejemplo # 2 de:
https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.11-K-Means.ipynb

#### Las técnicas no lineal (t-SNE, mapas autoorganizados...) son herramientas populares para la visualización en el espacio 2D y son útiles para la exploración/investigación de datos. Sin embargo, tienen parámetros ajustables que no son fáciles de ajustar y son difíciles de interpretar.

#### Feliz de seleccionar algunas referencias/material de lectura si alguien está interesado.

Por ejemplo en inglés:

https://scikit-learn.org/stable/modules/manifold.html#manifold

o

https://www.superdatascience.com/blogs/the-ultimate-guide-to-self-organizing-maps-soms

